In [0]:
'''
PROBLEM STATEMENT

Build a data pipeline with the files supplied to be served in an relational schema format.
Source : Could be any file system, for the purpose of this POC , I'm keeping the files in local storage(DBFS).
Target : Would be a relational database, I need to create a connection to that database in this POC.
'''

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType,DoubleType

schema_location = StructType([StructField("sr_no", IntegerType(), True)
                              ,StructField("id", IntegerType(), True)
                              ,StructField("wgs84_polygon", StringType()
                                           ,True)])
                              
schema_reservations = StructType([StructField("sr_no", IntegerType(), True)
                                  ,StructField("id", IntegerType(), True)
                                  ,StructField("customer_id", StringType(), True)
                                  ,StructField("start_latitude", StringType(), True)
                                  ,StructField("start_longitude", StringType(), True)
                                  ,StructField("srid", StringType(), True)
                                  ,StructField("net_price", DoubleType(),True)
                                 ])

In [0]:
df_loc = spark.read.format("csv").options(header = 'true').schema(schema_location).load("/FileStore/tables/felyxData/assignment_locations.csv")
df_resv = spark.read.format("csv").options(header = 'true').schema(schema_reservations).load("/FileStore/tables/felyxData/assignment_reservations.csv")

In [0]:
jdbcUrl = "jdbc:sqlserver://mysqlserver057.database.windows.net:1433;database=mysqldb"
connectionProperties = {
  "user" : "ashish",
  "password" : "******",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

try:
  df_loc.write.jdbc(url=jdbcUrl, table="dbo.location",mode = "overwrite", properties=connectionProperties )
except ValueError as error :
  print("Connector write failed", error)
    
try:
  df_resv.write.jdbc(url=jdbcUrl, table="dbo.reservation",mode = "overwrite", properties=connectionProperties )
except ValueError as error :
  print("Connector write failed", error)